In [117]:
# libraries
import requests        
from bs4 import BeautifulSoup
import xlwt
import xlrd
from xlutils.copy import copy
import random
from fake_useragent import UserAgent

In [118]:
# Enter the url of catalog page 
# Capture the data of catalog page which contains a lot of apps
def get_detail_page_url(url,headers):
    strhtml=requests.get(url = url,headers = headers)#fack header and avoid website redirects
    soup=BeautifulSoup(strhtml.text,'lxml')
    data = soup.select('#fcxH9b>div.WpDbMd>c-wiz>div>div.N4FjMb>div>c-wiz>c-wiz>c-wiz>div>div.ZmHEEd>div>c-wiz>div>div>div.RZEgze>div>div>div.bQVA0c>div>div>div.b8cIId.ReQCgd.Q9MA7b>a')
    # Establish the urls of every app' detail page
    s='http://play.google.com'
    url_all=[]
    for item in data:
        res = s+item.get('href')
        url_all.append(res)
    return url_all


# Use the urls to visit every app's detail page to get info
def get_detail(urlx,headers):   
    url = urlx
    strhtml = requests.get(url = url,headers = headers)#fack header and avoid website redirects
    soup = BeautifulSoup(strhtml.text,'html.parser')
    detail = []
    #package 
    detail.append(url[45:]) # http://play.google.com/store/apps/details?id=
    #name 
    detail.append(soup.find('h1',class_ = 'AHFaub').text) 
    #rank
    detail.append(soup.find('div', role = 'img')['aria-label'])#obtain the value of attribute
    #Ndownload
    detail.append(soup.find_all('span',class_ = 'htlgb')[4].text) #too many class htlgb  ctrl+f search for span.htlgb and find it's 4th
    #company
    detail.append(soup.find('span',class_ = 'T32cc UAO9ie').text) #too many class htlgb ctrl+f search for span.htlgb and find it's 22th
    #catalog
    detail.append(soup.find('a', class_ = 'hrTbp R8zArc',itemprop = "genre").text) #ues itemprop,class,<a>to locate content
    return detail

In [119]:
# Avoid detection of googles, fake web crawler header and user agent
def fack_headers():
    ua=UserAgent()
    headers={
        'User-Agent':ua.random}
    return headers

In [127]:
# Excel creation and initial excel
def create_excel(excel_name):
    wb = xlwt.Workbook()
    sh1 = wb.add_sheet('Info_Collection')
    sh1.write(0,0,'package')
    sh1.write(0,1,'name')
    sh1.write(0,2,'rank')
    sh1.write(0,3,'Ndownload')
    sh1.write(0,4,'company')
    sh1.write(0,5,'catalog')
    wb.save(excel_name+'.xls')
    

# Input data
def write(detail,i,excel_name):
    rb = xlrd.open_workbook(excel_name)
    r_sh = rb.sheet_by_name('Info_Collection')
    wb = copy(rb)
    sh1 = wb.get_sheet(0)
    i += 1
    sh1.write(i,0,detail[0])
    sh1.write(i,1,detail[1])
    sh1.write(i,2,detail[2])
    sh1.write(i,3,detail[3])
    sh1.write(i,4,detail[4])
    sh1.write(i,5,detail[5])
    wb.save(excel_name+'.xls')


In [128]:
def main():
    # Entre the url of catalog!!! and the name of excel!!!
    url='http://play.google.com/store/search?q=Children%20Monitor&c=apps&gl=FR'
    excel_name = 'Monitor app and children app'
    
    # Create an excel to store data
    create_excel(excel_name)
    # Obtain the app's detail page 
    url_all = get_detail_page_url(url,fack_headers())
    # loop to get detail of each app
    detail=[]
    for i in range (0,len(url_all)):
        detail=[]
        detail = get_detail(url_all[i],fack_headers())
        write(detail,i,excel_name)
    return 0

In [129]:
main()

0